# Исследование надежности заемщиков.

## Описание проекта
>Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. 
Входные данные от банка — статистика о платёжеспособности клиентов.
Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, 
которая оценивает способность потенциального заёмщика вернуть кредит банку.


**Цель исследования** — проверить 4 гипотезы:
1. Есть ли зависимость между количеством детей и возвратом кредита в срок?
2. Есть ли зависимость между семейным положением и возвратом кредита в срок?
3. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
4. Как разные цели кредита влияют на его возврат в срок?

**Ход исследования**

Данные о поведении пользователей мы получаем из файла data.csv. О качестве данных ничего не известно. Поэтому перед проверкой гипотез понадобится обзор данных. 

Мы проверим данные на ошибки и оценим их влияние на исследование. Затем, на этапе предобработки мы поищем возможность исправить самые критичные ошибки данных.
 
Таким образом, исследование пройдёт в три этапа:
 1. Обзор данных.
 2. Предобработка данных.
 3. Проверка гипотез.
 
**Описание данных**
- children — количество детей в семье
- days_employed — общий трудовой стаж в днях
- dob_years — возраст клиента в годах
- education — уровень образования клиента
- education_id — идентификатор уровня образования
- family_status — семейное положение
- family_status_id — идентификатор семейного положения
- gender — пол клиента
- income_type — тип занятости
- debt — имел ли задолженность по возврату кредитов
- total_income — ежемесячный доход
- purpose — цель получения кредита

### Шаг 1. Обзор данных

In [1]:
# импорт библиотеки pandas
import pandas as pd

Читаем файл data.csv из папки /datasets и сохраняем его в переменной df

In [2]:
df = pd.read_csv('/datasets/data.csv')

Выводим на экран первые десять строк таблицы:

In [3]:
# получение первых 10 строк таблицы df
display(df.head(10))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Одной командой получаем общую информацию о таблице:

In [4]:
# получаем общую информацию о данных в таблице df
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Наблюдения:**
1. Пропущенные значения в столбцах 'days_employed' и 'total_income'.
2. Столбцам 'days_employed' и 'total_income' стоит присвоить целочисленные значения(int)
3. Отрицательные значения в столбце 'days_employed'
4. Прыгающий регистр букв в столбце 'education'

### Шаг 2.1 Заполнение пропусков

In [5]:
# подсчёт пропусков
display(df.isna().sum())

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [6]:
procent_of_null = df.isnull().sum()/len(df)
procent_of_null = procent_of_null.sort_values(ascending=False)
display(procent_of_null*100)

days_employed       10.099884
total_income        10.099884
children             0.000000
dob_years            0.000000
education            0.000000
education_id         0.000000
family_status        0.000000
family_status_id     0.000000
gender               0.000000
income_type          0.000000
debt                 0.000000
purpose              0.000000
dtype: float64

Убедились, что пропуски дейстительно только в 'days_employed' и 'total_income'.
Возможно, эти люди никогда не работали или работали неофициально. К сожалению на данном этапе уточнить причину не представляется возможным, поэтому заменим пропуски медийными значениями, так как доля пропусков к общему количеству некритична

### Шаг 2.2 Проверка данных на аномалии и исправления.

В наблюдениях выше мы отмечали, так же отрицательное значение данных в столбце 'days_employed', исправим данную аномалию

In [7]:
#заменим отрицательные числа на положительные
def minus_on_plus(days):
    if days < 0:
        days *= -1
        return days
    else:
        return days  
df['days_employed'] = df['days_employed'].apply(minus_on_plus)
display(df['days_employed'].head(10))

0      8437.673028
1      4024.803754
2      5623.422610
3      4124.747207
4    340266.072047
5       926.185831
6      2879.202052
7       152.779569
8      6929.865299
9      2188.756445
Name: days_employed, dtype: float64

In [8]:
# Заменим пропуски на медианные значения.
total_income_median = df['total_income'].median()
days_employed_median = df['days_employed'].median()
df['days_employed'] = df['days_employed'].fillna(days_employed_median)
df['total_income'] = df['total_income'].fillna(total_income_median)

#повторный подсчёт пропусков
display(df.isna().sum())

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Шаг 2.3. Изменение типов данных.

Столбцам 'days_employed' и 'total_income' стоит присвоить целочисленные значения(int)

In [9]:
#Заменим тип данных методом astype
df['days_employed'] = df['days_employed'].astype('int')
df['total_income'] = df['total_income'].astype('int')

#проверим замену типа
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21525 non-null  int64 
 1   days_employed     21525 non-null  int64 
 2   dob_years         21525 non-null  int64 
 3   education         21525 non-null  object
 4   education_id      21525 non-null  int64 
 5   family_status     21525 non-null  object
 6   family_status_id  21525 non-null  int64 
 7   gender            21525 non-null  object
 8   income_type       21525 non-null  object
 9   debt              21525 non-null  int64 
 10  total_income      21525 non-null  int64 
 11  purpose           21525 non-null  object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


### Шаг 2.4. Удаление дубликатов.

Проверим наличие и количество дубликатов

In [10]:
df.duplicated().sum()

54

Возможно, что причина в их появлении банальное задвоение данных либо ошибочное создание дублей.
Удалим их

In [11]:
#Удалим дубли и сбросим индекс
df = df.drop_duplicates().reset_index(drop=True)

In [12]:
#Проверим
df.duplicated().sum()

0

Помимо явных дубликатов мы помним, что в столбце 'education' мы имеем прыгающий регистр букв. Устраним это.

In [13]:
#приведем в единый формат столбец 'education'
df['education'] = df['education'].str.lower()
display(df['education'].head(20))

0                  высшее
1                 среднее
2                 среднее
3                 среднее
4                 среднее
5                  высшее
6                  высшее
7                 среднее
8                  высшее
9                 среднее
10                 высшее
11                среднее
12                среднее
13    неоконченное высшее
14                 высшее
15                среднее
16                среднее
17                 высшее
18                среднее
19                среднее
Name: education, dtype: object

Проверим остальные столбцы на неявные дубликаты или иные аномалии.

In [14]:
#пройдем сначала по столбцу с детьми
df['children'].value_counts()

 0     14107
 1      4809
 2      2052
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Замечаем странные значения, -1 и 20. Думаю -1 заменим на 1, а 20 заменим на 2, так как исходя из здравого смысла эта опечатка могла случиться вероятнее всего, но и в целом количество таких строк не критично и на дальнейшие исследования повлияет не сильно.

In [15]:
# Заменим -1 на 1 и 20 на 2
df['children'] = df['children'].replace(-1, 1)
df['children'] = df['children'].replace(20, 2)

#проверим столбец еще раз
df['children'].value_counts()

0    14107
1     4856
2     2128
3      330
4       41
5        9
Name: children, dtype: int64

In [16]:
# Посмотрим на столбец с возрастом
df['dob_years'].value_counts()

35    616
40    607
41    606
34    601
38    597
42    596
33    581
39    572
31    559
36    554
44    545
29    544
30    538
48    537
37    536
50    513
43    512
32    509
49    508
28    503
45    497
27    493
52    484
56    484
47    477
54    476
46    473
53    459
57    456
58    456
51    448
59    443
55    443
26    408
60    374
25    357
61    354
62    349
63    269
24    264
64    262
23    253
65    194
22    183
66    182
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

В целом данные корректные, но присутсвуют строки с возрастом 0

In [17]:
#посмотрим на семейное положение
df['family_status'].value_counts()

женат / замужем          12344
гражданский брак          4163
Не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64

С семейным положением все в норме, идем дальше

In [18]:
#Смотрим на пол
df['gender'].value_counts()

F      14189
M       7281
XNA        1
Name: gender, dtype: int64

Одна строка содержащая странные данные на исследование не повлияет

In [19]:
#Смотрим на тип занятости
df['income_type'].value_counts()

сотрудник          11091
компаньон           5080
пенсионер           3837
госслужащий         1457
безработный            2
предприниматель        2
студент                1
в декрете              1
Name: income_type, dtype: int64

С типом занятости все в норме, остальные столбцы выглядят в норме либо на дальнейшие исследования не повлияют, общую предобработку данных в целом можно считать выполненной.

### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

Создаем два новых датафрейма, в которых:
каждому уникальному значению из education соответствует уникальное значение education_id — в первом;
каждому уникальному значению из family_status соответствует уникальное значение family_status_id — во втором.

In [20]:
#создаем два новых датафрейма
education_all = df[['education', 'education_id']]
family_status_all = df[['family_status', 'family_status_id']]

In [21]:
#Удаляем из исходного датафрейма столбцы education и family_status
df.drop(columns = ['education', 'family_status'],axis = 1)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,1,1,F,пенсионер,0,158616,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...
21466,1,4529,43,1,1,F,компаньон,0,224791,операции с жильем
21467,0,343937,67,1,0,F,пенсионер,0,155999,сделка с автомобилем
21468,1,2113,38,1,1,M,сотрудник,1,89672,недвижимость
21469,3,3112,38,1,0,M,сотрудник,1,244093,на покупку своего автомобиля


### Шаг 2.6. Категоризация дохода.

На основании диапазонов, указанных ниже, создаем столбец total_income_category с категориями:
1. 0–30000 — 'E';
2. 30001–50000 — 'D';
3. 50001–200000 — 'C';
4. 200001–1000000 — 'B';
5. 1000001 и выше — 'A'.

In [22]:
#Создадим функцию
def total_income_category(row):
    if row['total_income'] <= 30000:
        return 'E'
    elif 30001 < row['total_income'] <= 50000:
        return 'D'
    elif 50001 < row['total_income'] < 200000:
        return 'C'
    elif 200001 < row['total_income'] < 1000000:
        return 'B'
    else:
        return 'A'
df['total_income_category'] = df.apply(total_income_category, axis=1)
display(df.head())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,B
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,C
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,C
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,B
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,C


### Шаг 2.7. Категоризация целей кредита.

In [23]:
#Напишем функцию для точного отображения целей кредита
def purpose_category(row):
    if 'образован' in row['purpose']:
             return 'Получение образования'
    if 'авто' in row['purpose']:
             return 'Операции с автомобилем'
    if 'свадь' in row['purpose']:
             return 'Проведение свадьбы'
    if 'жиль' in row['purpose'] or 'недв' in row['purpose']:
             return 'Операции с недвижимостью'
    return 'Без категории'
df['purpose_category'] = df.apply(purpose_category, axis=1)
display(df.head())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,B,Операции с недвижимостью
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,C,Операции с автомобилем
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,C,Операции с недвижимостью
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,B,Получение образования
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,C,Проведение свадьбы


### Ответы на вопросы.

##### Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [24]:
#Создадим сводную таблицу по интересущим нас данным для наглядности
#Добавим столбец 'result', в котором отобразим отношение одной и той же категории людей, которые либо имели либо не имели задолжность
#Отсортируем по уменьшению
children_pivot = df.pivot_table(index = ['children'], columns = ['debt'], values = 'purpose_category', aggfunc='count')
children_pivot['result'] = children_pivot[1] / (children_pivot[0]+ children_pivot[1]) *100

display(children_pivot.sort_values('result', ascending = False))

debt,0,1,result
children,,,
4,37.0,4.0,9.756098
2,1926.0,202.0,9.492481
1,4411.0,445.0,9.163921
3,303.0,27.0,8.181818
0,13044.0,1063.0,7.535266
5,9.0,NaN,NaN


#### Вывод:
С увеличением количества детей просматривается увеличение количества просроченных задолженностей, несмотря на это, люди с 3 детьми чаще платят в срок чем люди с 1 ребенком. Возможно, нужна более крупная выборка для получения более детальных и точных выводов. Так же мы можем отметить, что бездетные, как правило, реже просрачивают оплату по кредиту, чем люди с детьми.

##### Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [25]:
#Создадим сводную таблицу по интересущим нас данным для наглядности
#Добавим столбец 'result', в котором отобразим отношение одной и той же категории людей, которые либо имели либо не имели задолжность
#Отсортируем по уменьшению
family_status_pivot = df.pivot_table(index=['family_status'], columns=['debt'], values='total_income_category', aggfunc='count')
family_status_pivot['result'] = family_status_pivot[1] / (family_status_pivot[0]+family_status_pivot[1]) * 100
display(family_status_pivot.sort_values('result', ascending = False))

debt,0,1,result
family_status,,,
Не женат / не замужем,2536,274,9.750890
гражданский брак,3775,388,9.320202
женат / замужем,11413,931,7.542126
в разводе,1110,85,7.112971
вдовец / вдова,896,63,6.569343


#### Вывод:
Семейный статус влияет на вероятность платежей по кредиту в срок - люди в разводе чаще плятят в срок чем те, кто не бывали в браке. А разведенные и овдовевшие платят в срок чаще, чем люди в браке.

##### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [26]:
#Создадим сводную таблицу по интересущим нас данным для наглядности
#Добавим столбец 'result', в котором отобразим отношение одной и той же категории людей, которые либо имели либо не имели задолжность
#Отсортируем по уменьшению
total_income_pivot = df.pivot_table(index=['total_income_category'], columns=['debt'], values='education_id', aggfunc='count')
total_income_pivot['result'] = total_income_pivot[1] / (total_income_pivot[0]+total_income_pivot[1]) * 100
display(total_income_pivot.sort_values('result', ascending = False))

debt,0,1,result
total_income_category,,,
E,20,2,9.090909
C,14673,1360,8.482505
A,24,2,7.692308
B,4684,356,7.063492
D,329,21,6.000000


#### Вывод:
Зависимости между уровнем уровнем дохода и возвратом кредита в срок нет.

##### Как разные цели кредита влияют на его возврат в срок?

In [27]:
#Создадим сводную таблицу по интересущим нас данным для наглядности
#Добавим столбец 'result', в котором отобразим отношение одной и той же категории людей, которые либо имели либо не имели задолжность
#Отсортируем по уменьшению
purpose_category_pivot = df.pivot_table(index=['purpose_category'], columns=['debt'], values='education_id', aggfunc='count')
purpose_category_pivot['result'] = purpose_category_pivot[1] / (purpose_category_pivot[0]+purpose_category_pivot[1]) * 100
display(purpose_category_pivot.sort_values('result', ascending = False))

debt,0,1,result
purpose_category,,,
Операции с автомобилем,3905,403,9.354689
Получение образования,3644,370,9.217738
Проведение свадьбы,2149,186,7.965739
Операции с недвижимостью,10032,782,7.231367


#### Вывод:
Цели кредита влияют.
Свадьбы и недвижимость как цели влияют на срок возврата лучше всего.

## Общий вывод:

Есть различные зависимости между целями кредита, семейным положением, уровнем дохода и количеством детей в семье. Внимание следует уделить вдовам/вдовцам и людям в разводе — с ними могут риски по невозврату кредитов выше. Так же стоит обратить внимание на цели кредита, так как они тоже имеют влияние на возврат.

Гипотезы проверены, для некоторых вопросов потребуется более крупная выборка.

Из полученных результатов можем сделать два вывода:

-Люди берущие кредит с оформленными отношениями (или которые в прошлом были в браке) и не имеющие детей - самые ответственные заемщики;

-Люди берущие кредит и состоящие в неофициальном браке или находящиеся без отношений, при этом имеющие 1 или 2 детей -  самые безответственные заемщики.

Общий портрет клиента с наибольшей вероятностью просрочки: не женат либо с неофициально оформленными отношениями, имеющий 1, 2 или 4 детей, с доходом до 30000 тысяч и с целью покупки авто 